# 06_04: A yearly top ten

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
allyears = pd.read_feather('allyears.feather')

In [ ]:
allyears_byyear = allyears.set_index(['sex', 'year']).sort_index()

In [ ]:
allyears_byyear.loc['M', 2000].sort_values('number', ascending=False)

In [ ]:
allyears_byyear.loc['M', 2023].sort_values('number', ascending=False).head(10)

In [ ]:
allyears_byyear.loc['F', 2023].sort_values('number', ascending=False).head(10)

In [ ]:
allyears_byyear.loc['F', 2023].sort_values('number', ascending=False).head(10).reset_index().name

In [ ]:
def getyear(sex, year):
    return (allyears_byyear.loc[sex, year].sort_values('number', ascending=False).head(10).reset_index().name)

In [ ]:
pd.DataFrame({year: getyear('M', year) for year in range(2014, 2024)})

In [ ]:
pd.DataFrame({year: getyear('F', year) for year in range(2014, 2024)})

In [ ]:
allyears_byname = allyears.set_index(['sex', 'name', 'year']).sort_index()

In [ ]:
pp.figure(figsize=(9,2))

for name in getyear('F', 2023):
    pp.plot(allyears_byname.loc['F', name], label=name)

pp.axis(xmin=1880, xmax=2024)
pp.legend(loc='upper left', fontsize=8);

In [ ]:
pp.figure(figsize=(9,2))

for name in getyear('M', 2023):
    pp.plot(allyears_byname.loc['M', name], label=name)

pp.axis(xmin=1880, xmax=2024)
pp.legend(loc='upper left', fontsize=8);

In [ ]:
alltime_f = allyears_byyear.loc['F'] \
                            .groupby('name') \
                            .sum() \
                            .sort_values('number', ascending=False) \
                            .head(10)

In [ ]:
alltime_f

In [ ]:
allyears_byname.loc[pd.IndexSlice['F', getyear('F', 2023), :]].groupby('name').sum()

In [ ]:
pp.figure(figsize=(9,2))

for name in alltime_f.index:
    pp.plot(allyears_byname.loc['M', name], label=name)

pp.axis(xmin=1880, xmax=2024)
pp.legend(loc='upper left', fontsize=8);